In [ ]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
from sdlnano import BayesianOptimizer_Auto
from calculation import BatchCalculation

In [ ]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [4]:
iteration = BayesianOptimizer_Auto.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 4


# Check the drug

In [5]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [6]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = BayesianOptimizer_Auto.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 02-11 15:11:03] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-11 15:11:15] ax.service.ax_client: Generated new trial 64 with parameters {'Drug_MW': 0.354, 'Drug_LogP': 0.391, 'Drug_TPSA': 0.067, 'Drug': 1.0, 'SL_1': 0.0, 'SL_2': 0.0, 'SL_3': 0.064656, 'LL_1': 0.0, 'LL_2': 1.0, 'LL_3': 0.0, 'P_1': 0.12033, 'P_2': 0.512004, 'P_3': 0.926952, 'S_1': 0.0, 'S_2': 1.0, 'S_3': 1.0, 'Water': 1.0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.concat(dfs, ax

4 min


In [ ]:
#unlabeled_trial.loc[unlabeled_trial['trial_index'] == 76, 'Water'] = 1


In [ ]:
# process the trails into ratios

unlabeled_trial_processed = BayesianOptimizer_Auto.process_trails(unlabeled_trial)
unlabeled_trial_processed

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,S_3,Water,Solu,Solu_STD,Size,Size_STD,PDI,PDI_STD,Complexity,Complexity_STD
0,64,354.0,3.91,67.0,0.276,0.000,0.0,0.018,0.000,0.276,...,0.333,0.333,None,None,None,None,None,None,None,None
1,65,354.0,3.91,67.0,0.208,0.000,0.0,0.037,0.176,0.177,...,0.000,0.266,None,None,None,None,None,None,None,None
2,66,354.0,3.91,67.0,0.283,0.000,0.0,0.000,0.000,0.283,...,0.391,0.391,None,None,None,None,None,None,None,None
3,67,354.0,3.91,67.0,0.210,0.000,0.0,0.000,0.198,0.235,...,0.000,0.129,None,None,None,None,None,None,None,None
4,68,354.0,3.91,67.0,0.219,0.000,0.0,0.022,0.132,0.219,...,0.403,0.237,None,None,None,None,None,None,None,None
5,69,354.0,3.91,67.0,0.243,0.000,0.0,0.000,0.096,0.243,...,0.417,0.166,None,None,None,None,None,None,None,None
6,70,354.0,3.91,67.0,0.251,0.000,0.0,0.039,0.000,0.251,...,0.333,0.333,None,None,None,None,None,None,None,None
7,71,354.0,3.91,67.0,0.213,0.000,0.0,0.073,0.213,0.141,...,0.000,0.139,None,None,None,None,None,None,None,None
8,72,354.0,3.91,67.0,0.247,0.000,0.0,0.000,0.089,0.247,...,0.500,0.500,None,None,None,None,None,None,None,None
9,73,354.0,3.91,67.0,0.213,0.007,0.0,0.016,0.193,0.176,...,0.000,0.000,None,None,None,None,None,None,None,None


In [26]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000,16.000,16.000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000
mean,71.500000,0.354,0.391,0.067,0.943020,0.001619,0.0,0.090501,0.643510,0.934021,0.0,0.105950,0.855395,0.679771,0.0,0.604244,0.500000,0.496616
std,4.760952,0.000,0.000,0.000,0.103446,0.006476,0.0,0.119260,0.378875,0.123244,0.0,0.154703,0.173933,0.214194,0.0,0.393004,0.516398,0.390721
min,64.000000,0.354,0.391,0.067,0.708007,0.000000,0.0,0.000000,0.000000,0.662557,0.0,0.000000,0.512004,0.247071,0.0,0.000000,0.000000,0.000000
25%,67.750000,0.354,0.391,0.067,0.937998,0.000000,0.0,0.000000,0.385838,0.952553,0.0,0.000000,0.756503,0.582229,0.0,0.234854,0.000000,0.211333
50%,71.500000,0.354,0.391,0.067,1.000000,0.000000,0.0,0.030003,0.771968,1.000000,0.0,0.000000,0.924871,0.682151,0.0,0.665726,0.500000,0.446944
75%,75.250000,0.354,0.391,0.067,1.000000,0.000000,0.0,0.158748,0.972490,1.000000,0.0,0.210513,1.000000,0.830253,0.0,1.000000,1.000000,1.000000
max,79.000000,0.354,0.391,0.067,1.000000,0.025903,0.0,0.342599,1.000000,1.000000,0.0,0.446966,1.000000,1.000000,0.0,1.000000,1.000000,1.000000


In [ ]:
# Convert ratios to volumes

transfer = BatchCalculation.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000
mean,71.500000,354.0,3.91,67.0,78.203125,0.153125,0.0,6.890625,51.362500,77.809375,0.0,8.246875,70.590625,56.765625,0.0,473.000000,222.500000,304.375000,-0.021875
std,4.760952,0.0,0.00,0.0,10.746743,0.612500,0.0,8.644097,29.886393,13.880324,0.0,11.864239,14.166842,20.856092,0.0,342.308048,242.296513,243.734521,0.298869
min,64.000000,354.0,3.91,67.0,62.650000,0.000000,0.0,0.000000,0.000000,49.350000,0.0,0.000000,49.350000,21.700000,0.0,0.000000,0.000000,0.000000,-0.700000
25%,67.750000,354.0,3.91,67.0,72.362500,0.000000,0.0,0.000000,32.987500,67.112500,0.0,0.000000,60.375000,45.587500,0.0,304.000000,0.000000,136.500000,-0.087500
50%,71.500000,354.0,3.91,67.0,75.250000,0.000000,0.0,2.800000,64.225000,79.450000,0.0,0.000000,72.975000,50.400000,0.0,388.500000,166.500000,298.500000,0.000000
75%,75.250000,354.0,3.91,67.0,85.400000,0.000000,0.0,13.125000,70.612500,88.025000,0.0,15.925000,82.687500,72.100000,0.0,765.750000,406.500000,385.750000,0.087500
max,79.000000,354.0,3.91,67.0,99.050000,2.450000,0.0,25.550000,88.550000,99.050000,0.0,34.650000,91.350000,91.000000,0.0,1000.000000,669.000000,1000.000000,0.350000


In [28]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       1.3
SL_1       0.0
SL_2       0.0
SL_3       0.1
LL_1       0.8
LL_2       1.2
LL_3       0.0
P_1        0.1
P_2        1.1
P_3        0.9
S_1        0.0
S_2        7.6
S_3        3.6
Water      4.9
Solvent   -0.0
dtype: float64

In [29]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [30]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.drop(['Drug_MW', 'Drug_LogP', 'Drug_TPSA'], axis=1).to_excel(transfer_path, index = False)

[INFO 02-11 15:20:35] ax.service.ax_client: Saved JSON-serialized state of optimization to `optimizer/optimizer_design_4.json`.


# Generate new protocol

In [31]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_.py'


In [32]:
# Update the script with Excel data
updated_script = BatchCalculation.update_transfer_script(template_protocol, transfer_path, output_script_path) 


In [33]:
updated_script

'protocol/iteration_4_OT_2_protocol_.py'

# Perform experiment

# Results

In [34]:
results = BatchCalculation.parameters_and_complexity(iteration)
results

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,64,354,3.91,67,0.276,0.000,0,0.018,0.000,0.276,0,0.033,0.141,0.256,0,0.333,0.333,0.333,7,0
1,65,354,3.91,67,0.208,0.000,0,0.037,0.176,0.177,0,0.045,0.214,0.143,0,0.734,0.000,0.266,7,0
2,66,354,3.91,67,0.283,0.000,0,0.000,0.000,0.283,0,0.000,0.174,0.260,0,0.217,0.391,0.391,5,0
3,67,354,3.91,67,0.210,0.000,0,0.000,0.198,0.235,0,0.000,0.235,0.124,0,0.871,0.000,0.129,5,0
4,68,354,3.91,67,0.219,0.000,0,0.022,0.132,0.219,0,0.061,0.168,0.179,0,0.360,0.403,0.237,8,0
5,69,354,3.91,67,0.243,0.000,0,0.000,0.096,0.243,0,0.000,0.176,0.243,0,0.417,0.417,0.166,6,0
6,70,354,3.91,67,0.251,0.000,0,0.039,0.000,0.251,0,0.099,0.141,0.218,0,0.333,0.333,0.333,7,0
7,71,354,3.91,67,0.213,0.000,0,0.073,0.213,0.141,0,0.047,0.185,0.128,0,0.861,0.000,0.139,7,0
8,72,354,3.91,67,0.247,0.000,0,0.000,0.089,0.247,0,0.000,0.214,0.202,0,0.000,0.500,0.500,5,0
9,73,354,3.91,67,0.213,0.007,0,0.016,0.193,0.176,0,0.000,0.261,0.135,0,1.000,0.000,0.000,7,0


In [35]:
size_raw = BatchCalculation.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,230.816958,0.161396
1,Good,A2,236.934684,0.215669
2,Caution,A3,222.392866,0.233377
3,Good,A4,209.282079,0.210330
4,Good,A5,206.699071,0.173871
5,Good,A6,194.706161,0.193144
6,Caution,B1,221.989342,0.230173
7,Good,B2,220.499103,0.183946
8,Good,B3,219.068536,0.222386
9,Good,B4,188.941688,0.180651


In [36]:
size_processed = BatchCalculation.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,233.875821,4.325886,0.188533,0.038377,1
1,Formulation 2,203.562437,7.777738,0.192449,0.018240,1
2,Formulation 3,219.783819,1.011563,0.203166,0.027181,1
3,Formulation 4,206.284721,15.538828,0.203432,0.023166,1
4,Formulation 5,202.536042,0.956216,0.180030,0.027483,1
5,Formulation 6,214.300418,6.384513,0.185892,0.035318,1
6,Formulation 7,216.798600,2.462949,0.185725,0.009820,1
7,Formulation 8,191.774764,9.542167,0.335650,0.027298,1
8,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,Formulation 10,184.576482,2.092070,0.232874,0.028548,1


In [37]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,64,354,3.91,67,0.276,0.000,0,0.018,0.000,0.276,...,0.333,0.333,7,0,Formulation 1,233.875821,4.325886,0.188533,0.038377,1
1,65,354,3.91,67,0.208,0.000,0,0.037,0.176,0.177,...,0.000,0.266,7,0,Formulation 2,203.562437,7.777738,0.192449,0.018240,1
2,66,354,3.91,67,0.283,0.000,0,0.000,0.000,0.283,...,0.391,0.391,5,0,Formulation 3,219.783819,1.011563,0.203166,0.027181,1
3,67,354,3.91,67,0.210,0.000,0,0.000,0.198,0.235,...,0.000,0.129,5,0,Formulation 4,206.284721,15.538828,0.203432,0.023166,1
4,68,354,3.91,67,0.219,0.000,0,0.022,0.132,0.219,...,0.403,0.237,8,0,Formulation 5,202.536042,0.956216,0.180030,0.027483,1
5,69,354,3.91,67,0.243,0.000,0,0.000,0.096,0.243,...,0.417,0.166,6,0,Formulation 6,214.300418,6.384513,0.185892,0.035318,1
6,70,354,3.91,67,0.251,0.000,0,0.039,0.000,0.251,...,0.333,0.333,7,0,Formulation 7,216.798600,2.462949,0.185725,0.009820,1
7,71,354,3.91,67,0.213,0.000,0,0.073,0.213,0.141,...,0.000,0.139,7,0,Formulation 8,191.774764,9.542167,0.335650,0.027298,1
8,72,354,3.91,67,0.247,0.000,0,0.000,0.089,0.247,...,0.500,0.500,5,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,73,354,3.91,67,0.213,0.007,0,0.016,0.193,0.176,...,0.000,0.000,7,0,Formulation 10,184.576482,2.092070,0.232874,0.028548,1


In [38]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,64,354,3.91,67,0.276,0.000,0,0.018,0.000,0.276,...,7,0,Formulation 1,233.875821,4.325886,0.188533,0.038377,1,552.0,0
1,65,354,3.91,67,0.208,0.000,0,0.037,0.176,0.177,...,7,0,Formulation 2,203.562437,7.777738,0.192449,0.018240,1,416.0,0
2,66,354,3.91,67,0.283,0.000,0,0.000,0.000,0.283,...,5,0,Formulation 3,219.783819,1.011563,0.203166,0.027181,1,566.0,0
3,67,354,3.91,67,0.210,0.000,0,0.000,0.198,0.235,...,5,0,Formulation 4,206.284721,15.538828,0.203432,0.023166,1,420.0,0
4,68,354,3.91,67,0.219,0.000,0,0.022,0.132,0.219,...,8,0,Formulation 5,202.536042,0.956216,0.180030,0.027483,1,438.0,0
5,69,354,3.91,67,0.243,0.000,0,0.000,0.096,0.243,...,6,0,Formulation 6,214.300418,6.384513,0.185892,0.035318,1,486.0,0
6,70,354,3.91,67,0.251,0.000,0,0.039,0.000,0.251,...,7,0,Formulation 7,216.798600,2.462949,0.185725,0.009820,1,502.0,0
7,71,354,3.91,67,0.213,0.000,0,0.073,0.213,0.141,...,7,0,Formulation 8,191.774764,9.542167,0.335650,0.027298,1,426.0,0
8,72,354,3.91,67,0.247,0.000,0,0.000,0.089,0.247,...,5,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
9,73,354,3.91,67,0.213,0.007,0,0.016,0.193,0.176,...,7,0,Formulation 10,184.576482,2.092070,0.232874,0.028548,1,426.0,0


In [39]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.233876,0.004326,0.276,0.0,0.188533,0.038377,0.583333
1,1,0.203562,0.007778,0.208,0.0,0.192449,0.018240,0.583333
2,1,0.219784,0.001012,0.283,0.0,0.203166,0.027181,0.416667
3,1,0.206285,0.015539,0.210,0.0,0.203432,0.023166,0.416667
4,1,0.202536,0.000956,0.219,0.0,0.180030,0.027483,0.666667
5,1,0.214300,0.006385,0.243,0.0,0.185892,0.035318,0.500000
6,1,0.216799,0.002463,0.251,0.0,0.185725,0.009820,0.583333
7,1,0.191775,0.009542,0.213,0.0,0.335650,0.027298,0.583333
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667
9,1,0.184576,0.002092,0.213,0.0,0.232874,0.028548,0.583333


In [40]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.233876,0.004326,0.276,0.0,0.188533,0.038377,0.583333
1,1,0.203562,0.007778,0.208,0.0,0.192449,0.018240,0.583333
2,1,0.219784,0.001012,0.283,0.0,0.203166,0.027181,0.416667
3,1,0.206285,0.015539,0.210,0.0,0.203432,0.023166,0.416667
4,1,0.202536,0.000956,0.219,0.0,0.180030,0.027483,0.666667
5,1,0.214300,0.006385,0.243,0.0,0.185892,0.035318,0.500000
6,1,0.216799,0.002463,0.251,0.0,0.185725,0.009820,0.583333
7,1,0.191775,0.009542,0.213,0.0,0.335650,0.027298,0.583333
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667
9,1,0.184576,0.002092,0.213,0.0,0.232874,0.028548,0.583333


In [41]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [42]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [43]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,64,354,3.91,67,0.276,0.0,0,0.018,0.000,0.276,...,0.583333,0,Formulation 1,0.233876,0.004326,0.188533,0.038377,1,0.276,0
1,65,354,3.91,67,0.208,0.0,0,0.037,0.176,0.177,...,0.583333,0,Formulation 2,0.203562,0.007778,0.192449,0.018240,1,0.208,0
2,66,354,3.91,67,0.283,0.0,0,0.000,0.000,0.283,...,0.416667,0,Formulation 3,0.219784,0.001012,0.203166,0.027181,1,0.283,0
3,67,354,3.91,67,0.210,0.0,0,0.000,0.198,0.235,...,0.416667,0,Formulation 4,0.206285,0.015539,0.203432,0.023166,1,0.210,0
4,68,354,3.91,67,0.219,0.0,0,0.022,0.132,0.219,...,0.666667,0,Formulation 5,0.202536,0.000956,0.180030,0.027483,1,0.219,0


In [44]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


[INFO 02-11 17:16:01] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [45]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [46]:
loaded_ax_client = BayesianOptimizer_Auto.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 02-11 17:16:02] ax.service.ax_client: Completed trial 64 with data: {'Solu': (0.276, 0), 'Size': (0.233876, 0.004326), 'PDI': (0.188533, 0.038377), 'Complexity': (0.583333, 0)}.
[INFO 02-11 17:16:02] ax.service.ax_client: Completed trial 65 with data: {'Solu': (0.208, 0), 'Size': (0.203562, 0.007778), 'PDI': (0.192449, 0.01824), 'Complexity': (0.583333, 0)}.
[INFO 02-11 17:16:02] ax.service.ax_client: Completed trial 66 with data: {'Solu': (0.283, 0), 'Size': (0.219784, 0.001012), 'PDI': (0.203166, 0.027181), 'Complexity': (0.416667, 0)}.
[INFO 02-11 17:16:02] ax.service.ax_client: Completed trial 67 with data: {'Solu': (0.21, 0), 'Size': (0.206285, 0.015539), 'PDI': (0.203432, 0.023166), 'Complexity': (0.416667, 0)}.
[INFO 02-11 17:16:02] ax.service.ax_client: Completed trial 68 with data: {'Solu': (0.219, 0), 'Size': (0.202536, 0.000956), 'PDI': (0.18003, 0.027483), 'Complexity': (0.666667, 0)}.
[INFO 02-11 17:16:02] ax.service.ax_client: Completed trial 69 with data: {'Solu': 